In [1]:
import scanpy as sc, scprep
from scipy.sparse import csr_matrix
import pandas as pd
import glob
import numpy as np

In [3]:
path_ds = 'raw/'
path_out = 'processed/'

# GSE101207

In [6]:
!wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE101nnn/GSE101207/matrix/GSE101207-GPL11154_series_matrix.txt.gz
!wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE101nnn/GSE101207/matrix/GSE101207-GPL17021_series_matrix.txt.gz

!gunzip GSE101207-GPL11154_series_matrix.txt.gz
!gunzip GSE101207-GPL17021_series_matrix.txt.gz

!mv GSE101207-GPL11154_series_matrix.txt raw
!mv GSE101207-GPL17021_series_matrix.txt raw

--2024-03-05 15:26:54--  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE101nnn/GSE101207/matrix/GSE101207-GPL11154_series_matrix.txt.gz
           => ‘GSE101207-GPL11154_series_matrix.txt.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /geo/series/GSE101nnn/GSE101207/matrix ... done.
==> SIZE GSE101207-GPL11154_series_matrix.txt.gz ... 3896
==> PASV ... done.    ==> RETR GSE101207-GPL11154_series_matrix.txt.gz ... done.
Length: 3896 (3.8K) (unauthoritative)

GSE101207-GPL11154_ 100%[===================>]   3.80K  --.-KB/s    in 0s      

2024-03-05 15:26:54 (345 MB/s) - ‘GSE101207-GPL11154_series_matrix.txt.gz’ saved [3896]

--2024-03-05 15:26:54--  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE101nnn/GSE101207/matrix/GSE101207-G

In [15]:
! wget https://ars.els-cdn.com/content/image/1-s2.0-S2211124719302141-mmc2.xlsx
! mv 1-s2.0-S2211124719302141-mmc2.xlsx raw

--2024-07-24 23:37:51--  https://ars.els-cdn.com/content/image/1-s2.0-S2211124719302141-mmc2.xlsx
Resolving ars.els-cdn.com (ars.els-cdn.com)... 172.64.155.214, 104.18.32.42, 2606:4700:4400::6812:202a, ...
Connecting to ars.els-cdn.com (ars.els-cdn.com)|172.64.155.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1947147 (1.9M) [application/excel]
Saving to: ‘1-s2.0-S2211124719302141-mmc2.xlsx’

1-s2.0-S22111247193 100%[===================>]   1.86M  --.-KB/s    in 0.04s   

2024-07-24 23:37:51 (49.2 MB/s) - ‘1-s2.0-S2211124719302141-mmc2.xlsx’ saved [1947147/1947147]



In [39]:
!wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE101nnn/GSE101207/suppl/GSE101207_RAW.tar
!mv GSE101207_RAW.tar raw
!tar -xvf raw/GSE101207_RAW.tar -C raw

--2024-03-05 15:45:47--  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE101nnn/GSE101207/suppl/GSE101207_RAW.tar
           => ‘GSE101207_RAW.tar’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.7, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /geo/series/GSE101nnn/GSE101207/suppl ... done.
==> SIZE GSE101207_RAW.tar ... 421345280
==> PASV ... done.    ==> RETR GSE101207_RAW.tar ... done.
Length: 421345280 (402M) (unauthoritative)

GSE101207_RAW.tar   100%[===================>] 401.83M   122MB/s    in 3.3s    

2024-03-05 15:45:52 (122 MB/s) - ‘GSE101207_RAW.tar’ saved [421345280]



In [ ]:
sm=pd.read_table(path_ds+'GSE101207-GPL11154_series_matrix.txt',skiprows=34,index_col=0)
sm1=pd.read_table(path_ds+'GSE101207-GPL17021_series_matrix.txt',skiprows=34,index_col=0)
print('SM1 is mouse:',(sm1.loc['!Sample_organism_ch1',:]=='Mus musculus').all())
print('SM shape:',sm.shape)

In [18]:
obs = pd.read_excel('raw', sheet_name='Donor_Full_info', skiprows=2, index_col=0)

In [20]:
obs

,Gender,Age,BMI,HbA1c,Race,Height,Weight,Death cause,disease status
Sample,,,,,,,,,
H1,M,27,20.60,5.400,Ascian-filipino,NaN,NaN,Stroke,non-diabetic
H2,M,21,22.80,5.200,Caucasian,69.0,154.0,Trauma,non-diabetic
H3,F,38,34.40,5.000,Caucasian,69.0,233.0,Anoxic event,non-diabetic
H4,M,52,22.00,5.600,Caucasian,74.0,173.0,Stroke,non-diabetic
H5,M,28,30.80,4.900,Caucasian,71.0,220.0,Stroke,non-diabetic
H6,M,44,34.60,0.054,Caucasian,60.0,255.0,automobile accident,non-diabetic
T2D1,M,58,39.30,8.900,Caucasian,70.0,274.0,Anoxic event,Type 2 diabetes
T2D2,M,61,28.10,5.200,Caucasian,NaN,NaN,Cerebral Vascular accident,Type 2 diabetes
T2D3,M,51,35.59,0.071,Hispanic,57.0,240.0,stroke,Type 2 diabetes


In [ ]:
obs=pd.read_excel(path_ds+'1-s2.0-S2211124719302141-mmc2.xlsx',sheet_name='Celltype.info',
                  skiprows=2,index_col=0)

In [25]:
# Drop unused columns
obs.drop(['Tx.all','Tx.D'],axis=1,inplace=True)
# rename ct col
obs.rename({'celltype':'cell_type','Donor':'donor'},axis=1,inplace=True)

In [27]:
# Parse cts
obs['cell_type']=obs['cell_type'].replace({'allBeta':'Beta'})
obs['cell_type_original']=obs['cell_type']
obs['cell_type']=obs['cell_type'].str.lower().replace({
    'duct':'ductal','psc':'stellate','pp':'gamma'}).fillna('dropped')

# From the paper: "28,026 “clean” cells without ambiguity" - others I have named here as dropped

In [28]:
donor_suf=set()
for idx,data in obs.iterrows():
    donor_suf.add(tuple([data['donor'],(idx.split('_')[1] if len(idx.split('_'))>1 else None)]))
print(donor_suf)

{('H5', 'e'), ('H1', None), ('T2D1', 'g'), ('H6', 'f'), ('H3', 'c'), ('H2', 'b'), ('H4', 'd'), ('T2D2', 'h'), ('T2D3', 'i')}


In [29]:
# Change suffix to donor to make processing easier downstream when mattching to expression
obs.index=[i.split('_')[0]+'_'+donor for i,donor in zip(obs.index,obs.donor)]

In [ ]:
# Donor info
donor_df=pd.read_excel(path_ds+'1-s2.0-S2211124719302141-mmc2.xlsx',sheet_name='Donor_Full_info',
                  skiprows=2,index_col=0)

In [62]:
# Edit colnames and values
donor_df.fillna('NA',inplace=True)
donor_df.drop(['Weight','Height'],axis=1,inplace=True)
donor_df.rename({'Gender':'sex','Age':'age','Race':'ethnicity',
                 'Death cause':'death_cause','disease status':'disease'},axis=1,inplace=True)
donor_df['sex'].replace({'M':'male','F':'female'},inplace=True)
donor_df['age']=donor_df['age'].apply(lambda x: str(x)+' y')
donor_df['ethnicity']=donor_df['ethnicity'].str.lower()
donor_df['death_cause']=donor_df['death_cause'].str.lower()
donor_df['disease'].replace(
    {'non-diabetic':'control','Type 2 diabetes':'T2D'},inplace=True)

In [64]:
# Add donor info to obs
for col in donor_df.columns:
    obs[col]=obs['donor'].map(donor_df[col].to_dict())

In [65]:
obs['organ']='pancreas'
obs['tissue']='pancreatic islets'
obs['organism']='human'

In [47]:
adata=[]
for donor in obs.donor.unique():
    files=glob.glob(path_ds+'*_'+donor+'.down.gene_exon_tagged.cleaned.dge.txt.gz')
    if len(files)!=1:
        raise ValueError('Not exactly 1 file per donor')
    x_sub=pd.read_table(files[0], index_col=0).T
    x_sub.index=[i+'_'+donor for i in x_sub.index]
    x_sub=sc.AnnData(x_sub)
    x_sub.X=csr_matrix(x_sub.X)
    adata.append(x_sub)
adata=sc.concat(adata,join='outer')

/tmp/ipykernel_872962/3906879862.py:8: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  x_sub=sc.AnnData(x_sub)
/tmp/ipykernel_872962/3906879862.py:8: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  x_sub=sc.AnnData(x_sub)
/tmp/ipykernel_872962/3906879862.py:8: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  x_sub=sc.AnnData(x_sub)
/tmp/ipykernel_872962/3906879862.py:8: FutureWarning: X.dtype be

In [83]:
# Add obs
adata.obs=obs.reindex(adata.obs_names)
# Fill ct info if not present
adata.obs['cell_type'].fillna('dropped',inplace=True)
# Save orginal X
adata.layers['raw']=adata.X.copy()

In [85]:
# normalise
sc.pp.normalize_total(adata)

/gpfs/gibbs/project/krishnaswamy_smita/av622/conda_envs/gspa/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:197: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))


In [86]:
# Sqrt normalize (to match our analysis)
sc.pp.sqrt(adata)

In [90]:
adata_subset = adata[(adata.obs['disease'].isin(['control','T2D'])) & (adata.obs['cell_type'] == 'beta')]